In [ ]:
from pyspark.sql import SparkSession
import requests
import json

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("Time_services").master("spark://spark-master_new:7077").config("spark.executor.memory", "512m").config("spark.eventLog.enabled", "true").config("spark.eventLog.dir", "file:///opt/workspace/events").getOrCreate()

# Read JSON files from HDFS
df = spark.read.json("all_papers.json")

In [ ]:
from datetime import datetime

In [ ]:
def transform_row(row):
    json_id = row['id']
    text = row['title']  # Choose 'abstract' or 'title'

    result = call_hugging_face_api(text)
    date_string = result['date'] 

    #cambiar formato
    date_object = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S")

    formatted_date = {
        "day": date_object.day,
        "month": date_object.month,
        "year": date_object.year,
        "weekday": date_object.strftime("%A")
    }

    formatted_date_json = json.dumps(formatted_date)

    return (json_id, formatted_date_json)

In [ ]:
transformed_data = df.rdd.map(transform_row)

transformed_df = transformed_data.toDF(["id", "formatted_date"])    

# Write the result to a CSV file
result_df.write.csv("output/papers_with_locations.csv", header=True)

In [ ]:
spark.stop()